In [2]:
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image Parameters
image_height = 224
image_width = 224
n_color_channels = 3

n_classes = 6


base_model = tf.keras.applications.Xception(input_shape=(image_height, image_width, n_color_channels),
                                            include_top=False,
                                            weights="imagenet")


base_model.trainable = False

# Model Parameters as requested
x = base_model.output
x = BatchNormalization()(x)
x = GlobalAveragePooling2D()(x)
x = Dense(8, activation='relu')(x)
x = Dropout(0.4)(x)
output = Dense(n_classes, activation='relu')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compiling the model
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Printing model summary
model.summary()

# Data directories
train_dir = '/content/NEU-DET/train/images'
validation_dir = '/content/NEU-DET/validation/images'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

# Generate training and validation data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='sparse'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='sparse'
)


epochs = 10
model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

83683744/83683744 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 111, 111, 32)         128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  

#Testing the model

In [5]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Test Image
img_path = '/content/NEU-DET/train/images/rolled-in_scale/rolled-in_scale_108.jpg'
img = image.load_img(img_path, target_size=(image_height, image_width))

# Converting the image to an array
img_array = image.img_to_array(img)

# Resize the image to match dimensions of model
img_array = np.expand_dims(img_array, axis=0)

# Normalising it
img_array = img_array / 255.0

predictions = model.predict(img_array)

# Get the predicted class
predicted_class = np.argmax(predictions)

class_label_dictionary = {0: "Crazing", 1: "Inclusion", 2: "Patches", 3: "Pitted_surface", 4: "Rolled-in_surface", 5: "Scratches"}

print("Predicted class:", class_label_dictionary[predicted_class])


1/1 [==============================] - 0s 23ms/step
Predicted class: Rolled-in_surface
